In [14]:
url1 = (r'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/latest/owid-covid-latest.json')

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
import requests, pandas as pd
from bs4 import BeautifulSoup
import urllib.request

In [67]:

#This is live data from the our world in data github. This is the csv they put on the website. Covid19 data.
data=pd.read_json(url1)
data1 = pd.DataFrame(data).T
data1 = data1.astype(str)
data1 = data1[data1 != 'None']
data1 = data1['total_deaths'].astype(float)


In [17]:
url2 = ('https://apps.who.int/gho/athena/api/GHO/HWF_0002.json?filter=YEAR:2019&filter=text')
url2

'https://apps.who.int/gho/athena/api/GHO/HWF_0002.json?filter=YEAR:2019&filter=text'

In [19]:
data2 = pd.read_csv(url2)

HTTPError: HTTP Error 403: Forbidden

In [30]:

data2 = pd.read_csv(r'C:\Users\vw191\OneDrive\Documents\GitHub\107SBakst.github.io\data\who_doctors.csv')

data2 = data2[data2.Period == 2019]
data2

,IndicatorCode,Indicator,ValueType,ParentLocationCode,ParentLocation,Location type,SpatialDimValueCode,Location,Period type,Period,...,FactValueUoM,FactValueNumericLowPrefix,FactValueNumericLow,FactValueNumericHighPrefix,FactValueNumericHigh,Value,FactValueTranslationID,FactComments,Language,DateModified
0,HWF_0001,"Medical doctors (per 10,000)",numeric,AFR,Africa,Country,TCD,Chad,Year,2019,...,NaN,NaN,NaN,NaN,NaN,0.54,NaN,NaN,EN,2021-02-17T00:00:00.000Z
1,HWF_0001,"Medical doctors (per 10,000)",numeric,AFR,Africa,Country,BEN,Benin,Year,2019,...,NaN,NaN,NaN,NaN,NaN,0.65,NaN,NaN,EN,2021-02-17T00:00:00.000Z
2,HWF_0001,"Medical doctors (per 10,000)",numeric,WPR,Western Pacific,Country,PNG,Papua New Guinea,Year,2019,...,NaN,NaN,NaN,NaN,NaN,0.66,NaN,NaN,EN,2021-02-17T00:00:00.000Z
3,HWF_0001,"Medical doctors (per 10,000)",numeric,AFR,Africa,Country,TGO,Togo,Year,2019,...,NaN,NaN,NaN,NaN,NaN,0.78,NaN,NaN,EN,2021-02-17T00:00:00.000Z
4,HWF_0001,"Medical doctors (per 10,000)",numeric,AFR,Africa,Country,MOZ,Mozambique,Year,2019,...,NaN,NaN,NaN,NaN,NaN,0.85,NaN,NaN,EN,2021-02-17T00:00:00.000Z
5,HWF_0001,"Medical doctors (per 10,000)",numeric,AFR,Africa,Country,SEN,Senegal,Year,2019,...,NaN,NaN,NaN,NaN,NaN,0.88,NaN,NaN,EN,2021-02-17T00:00:00.000Z
6,HWF_0001,"Medical doctors (per 10,000)",numeric,AFR,Africa,Country,BFA,Burkina Faso,Year,2019,...,NaN,NaN,NaN,NaN,NaN,0.94,NaN,NaN,EN,2021-02-17T00:00:00.000Z
7,HWF_0001,"Medical doctors (per 10,000)",numeric,AFR,Africa,Country,GHA,Ghana,Year,2019,...,NaN,NaN,NaN,NaN,NaN,1.06,NaN,NaN,EN,2021-02-17T00:00:00.000Z
8,HWF_0001,"Medical doctors (per 10,000)",numeric,AFR,Africa,Country,GMB,Gambia,Year,2019,...,NaN,NaN,NaN,NaN,NaN,1.07,NaN,NaN,EN,2021-02-17T00:00:00.000Z
9,HWF_0001,"Medical doctors (per 10,000)",numeric,AFR,Africa,Country,RWA,Rwanda,Year,2019,...,NaN,NaN,NaN,NaN,NaN,1.18,NaN,NaN,EN,2021-02-17T00:00:00.000Z
